In [1]:
!pip install huggingface_hub transformers peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 28.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
    DataCollatorForSeq2Seq, EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ 모델 및 토크나이저 로드
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.pad_token_id

# ✅ LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj","o_proj"]
)

# ✅ 모델에 LoRA 적용
model = get_peft_model(model, lora_config).to(device)

# ✅ 데이터 로드 (단일 JSON 파일)
dataset = load_dataset("json", data_files={"data": "QAdata.json"})["data"]

# ✅ 데이터셋 나누기 (80% train, 20% eval)
split_ratio = 0.8  # 원하는 비율로 조정 가능
split_index = int(len(dataset) * split_ratio)

train_dataset = dataset.select(range(split_index))  # 80% 학습 데이터
eval_dataset = dataset.select(range(split_index, len(dataset)))  # 20% 검증 데이터

# ✅ 데이터 전처리 함수 (labels 추가)
def preprocess_function(examples):
    inputs = [q for q in examples["question"]]   # 질문 데이터
    outputs = [a for a in examples["answer"]]    # 정답 데이터

    # 토크나이징 (패딩 처리)
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(outputs, padding="max_length", truncation=True, max_length=512)

    # labels 추가 (모델 학습에 필요)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ✅ 데이터셋 변환 (labels 추가됨)
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# ✅ Data Collator (패딩 자동 처리)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    save_total_limit=2,
    save_steps=100,
    load_best_model_at_end=True,      # 훈련 후 가장 좋은 모델 자동 로드
    metric_for_best_model="eval_loss", # 가장 좋은 모델 선택 기준 메트릭
    greater_is_better=False,          # 작은 값이 더 좋은 메트릭 (예: loss)
)

# ✅ Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator  
)

# ✅ 학습 시작
trainer.train()

# ✅ 학습된 모델 저장 (LoRA 어댑터)
model.save_pretrained("./fine_tuned_model_lora")


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

Generating data split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_305/112743749.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,2.031900,1.071625
2,0.977400,0.789000
3,0.713100,0.704366
4,0.641700,0.672438
5,0.646300,0.682227
6,0.608300,0.671681
7,0.562700,0.643345
8,0.573200,0.649569
9,0.536900,0.648167
10,0.516300,0.670862
